# Data Challenge

In [1]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [2]:
dataFiles = sorted(glob('Training/Small/Sample**.csv'))
dataFiles.extend(sorted(glob('Validation/Small/Sample**.csv')))
dataFiles.extend(sorted(glob('Training/Large/Sample**.csv')))
dataFiles.extend(sorted(glob('Validation/Large/Sample**.csv')))

In [3]:
#Criando dataFrames para cada arquivo
dataFrames = []

particleSizeData = pd.read_csv('Training/Training Operation Profiles of Samples.csv', ';')
particleSizeData.append(pd.read_csv('Validation/Validation Operation Profiles of Samples.csv', ';'))
for file in dataFiles:
    df = pd.read_csv(file)
    df["Pressure_Diferential(psi)"] = df["Upstream_Pressure(psi)"] - df["Downstream_Pressure(psi)"] #Calculo da perda de Carga
    df["sample"] = int(file[-6:-4])
    df.join(particleSizeData.set_index('sample'), on='sample')
    first_occurence = df[df['Pressure_Diferential(psi)']>20].index[0]
    df = df.iloc[:first_occurence]
    dataFrames.append(df)

In [4]:
#Criando coluna de vida util remanscente
for df in dataFrames:
    lastTime = df['Time(s)'].iloc[[-1]]
    df['Remaining_Lifespan(s)'] = [(lastTime.values[0] - instant) for instant in df['Time(s)']];

In [5]:
#Unindo dataframes em um único
dataFrame = pd.concat(dataFrames).reset_index();

In [6]:
#Definindo dataFrame de teste:
import numpy
randnums = [*numpy.random.randint(1,16,3), *numpy.random.randint(33,48,3)]
dataFrames_test = []
for sample in randnums:
    dataFrames_test.append(dataFrame.loc[dataFrame['sample'] == sample])
    
dataFrame_test = pd.concat(dataFrames_test).reset_index();
df_test = dataFrame_test.drop(columns=['sample', 'index', 'level_0'])
y_test = df_test['Remaining_Lifespan(s)']
df_test = df_test.loc[:, df_test.columns != 'Remaining_Lifespan(s)']

In [7]:
#Aplicando Modelo Random Forest

In [8]:
#Separando variáveis dependentes e independentes
#Variável dependente:
df = dataFrame.drop(columns=['sample', 'index'])
y_train = df['Remaining_Lifespan(s)']
X_train = df.loc[:, df.columns != 'Remaining_Lifespan(s)']

In [9]:
#Importando Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

#Criando obj Random Forest Regressor
RFReg = RandomForestRegressor(n_estimators = 10, random_state = 0)

#Fit the random forest regressor with training data represented by X_train and y_train
RFReg.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

In [10]:
#Teste
#Criando dataFrames para cada arquivo
y_predict_rfr = RFReg.predict(df_test)

In [11]:
from sklearn import metrics
r_square = metrics.r2_score(y_test,y_predict_rfr)

In [12]:
r_square

0.9814071313409877

In [14]:
y_predict_rfr

array([257.38, 240.91, 265.37, ...,   0.  ,   0.  ,   0.  ])